In [1]:
import pandas as pd
import pickle 
import sys
import os
sys.path.append("/Ray_finrl/findrl_ray/finenv")
sys.path.append("./FinRL")
import finrl
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
from finenv.save_model import ftpsavemodel
import psutil
import ray
from datetime import datetime
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility
from ray.rllib.agents import ppo
train = pd.read_csv('dataset/train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']
#os.environ["OMP_NUM_THREADS"] = "1"
#os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

Stock Dimension: 82, State Space: 821


In [2]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-2)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))

register_env("finrl", env_creator)

In [3]:
ray.shutdown()
#ray.init(num_cpus=122,dashboard_port=8080)
print(f"ray is being initialized")

ray is being initialized


In [4]:
from ray.rllib.algorithms.ddpg.ddpg import DDPGConfig
from ray import air
from ray import tune
config = DDPGConfig().framework(framework="torch").config["model"]["fcnet_hiddens"] = [256, 256, 256].config['train_batch_size'] = 10000
# Print out some default values.
print(config.lr)

#trainer = config.build(env="finrl") 
# Update the config object.

config = config.training(lr=tune.grid_search([0.001, 0.0001]))
# Set the config object's env.
config = config.environment(env="finrl")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "DDPG",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()
#Build trainer with above config.


AttributeError: 'DDPGConfig' object has no attribute 'config'

In [ ]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes = 10
agent_name = 'ppo'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
date = datetime.now().strftime('%y%m%d')

while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    rwd = results[-1]['episode_reward_mean']
    if ep % 5 == 0:
        print(f'Mean Rwd:{rwd}')
    if ep % 5 == 0:
        #cwd_checkpoint = "results/checkpoints/" +  + '_' + str(ep)
        cwd_checkpoint = f"model/{agent_name}_{date}"
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint{ep} saved in directory {cwd_checkpoint}")
bar.close()

In [ ]:
#Save latest ckpt point
cwd_checkpoint = f"results/org_{date}_{ep}"
trainer.save(cwd_checkpoint)
#Extract model weights 
model_weights = trainer.get_policy().get_weights()
print('passed model weights')
config2 = ppo.PPOConfig()
print('config created')
config2 = config2.environment(env_config={'hmax':500,'initial_amount':1000000})
config2 = config2.training(gamma=0.9, lr=0.005, kl_coeff=0.3)  
config2 = config2.rollouts(num_rollout_workers=0) 
config2 = config2.framework(framework="torch")
config2['seed'] = 42
config2["model"]["fcnet_hiddens"] = [256, 256, 256]
#config2['sgd_minibatch_size'] = 128
#config2['num_sgd_iter'] = 30
config2['train_batch_size'] = 50000
trainer2 = ppo.PPOTrainer(env='finrl', config=config2)
trainer2.get_policy().set_weights(model_weights)
print('New Weights loaded. ')
ckpt2 = f"{cwd_checkpoint}_wt"
trainer2.save(ckpt2)
zip_filename = f'ckpt_wt{date}_{ep}.zip'
savefile = ftpsavemodel(ckpt2,zip_filename)
print(f'file{savefile}in{ckpt2}')